In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
""" Neural Machine Translation.
    Seq to Seq model.
    translate English to Korean.

"""
# [batch, seq_len]
input_en = ['I PROPOSE to consider the question, ‘Can machines think?’ This should begin with definitions of the meaning of the terms ‘machine’ and ‘think’.',
            'A computer would deserve to be called intelligent if it could deceive a human into believing that it was human.',
            'Be alone, that is the secret of invention; be alone, that is when ideas are born.',
            'I don`t care that they stole my idea . . I care that they don`t have any of their own.',
            'If you can’t explain it to a six-year-old, you probably don’t understand it yourself',
            'I am enough of an artist to draw freely upon my imagination. Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.'
            ]

input_ko = ['나는 ‘기계들이 생각할 수 있을까?’라는 질문을 고려해볼 것을 제안한다. 이 질문은 `기계’와 `생각’이라는 용어의 의미에 대한 정의에서 시작해야 한다.',
            '만약 컴퓨터가 인간을 속여 자신을 마치 인간인 것처럼 믿게 할 수 있다면 컴퓨터를 ‘지능이 있는’ 이라고 부를만한 가치가 충분히 있다.',
            '혼자가 되는것, 그것이 발명의 비밀입니다. 혼자가 되는 것, 그것이 아이디어들이 탄생 할 때 입니다.',
            '나는 그들이 내 아이디어를 훔치는 것은 신경쓰지 않는다.. 나는 그들이 자기 자신의 것을 가지고 있지 않음을 걱정할뿐 이다.',
            '6살 아이에게 설명할 수 없다면, 당신은 분명 스스로 이해하지 못하고 있는 것이다.',
            '나는 내 상상력을 자유롭게 그릴 수 있는 충분한 예술가 입니다. 상상력은 지식보다 중요합니다. 지식은 제한되어 있습니다. 상상력은 세상을 에워쌉니다.'
            ]

for i in input_en:
  print(len(i))
  print(i)

for i in input_ko:
  print(len(i))
  print(i)


143
I PROPOSE to consider the question, ‘Can machines think?’ This should begin with definitions of the meaning of the terms ‘machine’ and ‘think’.
111
A computer would deserve to be called intelligent if it could deceive a human into believing that it was human.
81
Be alone, that is the secret of invention; be alone, that is when ideas are born.
86
I don`t care that they stole my idea . . I care that they don`t have any of their own.
84
If you can’t explain it to a six-year-old, you probably don’t understand it yourself
161
I am enough of an artist to draw freely upon my imagination. Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.
85
나는 ‘기계들이 생각할 수 있을까?’라는 질문을 고려해볼 것을 제안한다. 이 질문은 `기계’와 `생각’이라는 용어의 의미에 대한 정의에서 시작해야 한다.
75
만약 컴퓨터가 인간을 속여 자신을 마치 인간인 것처럼 믿게 할 수 있다면 컴퓨터를 ‘지능이 있는’ 이라고 부를만한 가치가 충분히 있다.
56
혼자가 되는것, 그것이 발명의 비밀입니다. 혼자가 되는 것, 그것이 아이디어들이 탄생 할 때 입니다.
69
나는 그들이 내 아이디어를 훔치는 것은 신경쓰지 않는다.. 나는 그들이 자기 자신의 것을 가지고 있지 않음을 걱정할뿐 이다.
46
6살 

In [3]:
def basic_tokenizer_space(sentence): # Basic tokenizer. split by space.
  tok_sentence = [[word for word in sentence[j].split(' ')] for j in range(len(sentence))]
  return tok_sentence

tok_input_en = basic_tokenizer_space(input_en)
tok_input_ko = basic_tokenizer_space(input_ko)
print(tok_input_en)
print(tok_input_ko)

[['I', 'PROPOSE', 'to', 'consider', 'the', 'question,', '‘Can', 'machines', 'think?’', 'This', 'should', 'begin', 'with', 'definitions', 'of', 'the', 'meaning', 'of', 'the', 'terms', '‘machine’', 'and', '‘think’.'], ['A', 'computer', 'would', 'deserve', 'to', 'be', 'called', 'intelligent', 'if', 'it', 'could', 'deceive', 'a', 'human', 'into', 'believing', 'that', 'it', 'was', 'human.'], ['Be', 'alone,', 'that', 'is', 'the', 'secret', 'of', 'invention;', 'be', 'alone,', 'that', 'is', 'when', 'ideas', 'are', 'born.'], ['I', 'don`t', 'care', 'that', 'they', 'stole', 'my', 'idea', '.', '.', 'I', 'care', 'that', 'they', 'don`t', 'have', 'any', 'of', 'their', 'own.'], ['If', 'you', 'can’t', 'explain', 'it', 'to', 'a', 'six-year-old,', 'you', 'probably', 'don’t', 'understand', 'it', 'yourself'], ['I', 'am', 'enough', 'of', 'an', 'artist', 'to', 'draw', 'freely', 'upon', 'my', 'imagination.', 'Imagination', 'is', 'more', 'important', 'than', 'knowledge.', 'Knowledge', 'is', 'limited.', 'Imagin

In [4]:
vocab_word2index = {'<PAD>': 0, '<BOS>': 1, '<EOS>': 2} # Begin of sentence: [BOS], End of sentence: [EOS]
vocab_index2word = {0: '<PAD>', 1: '<BOS>', 2: '<BOS>'}

def make_vocabulary(tok_sentence):
  cur_index = len(vocab_word2index)
  for s in tok_sentence:
    for w in s:
      if w not in vocab_word2index:
        vocab_word2index[w] = cur_index
        vocab_index2word[cur_index] = w
        cur_index += 1

make_vocabulary(tok_input_en)
make_vocabulary(tok_input_ko)
print(vocab_word2index)
print(vocab_index2word)

{'<PAD>': 0, '<BOS>': 1, '<EOS>': 2, 'I': 3, 'PROPOSE': 4, 'to': 5, 'consider': 6, 'the': 7, 'question,': 8, '‘Can': 9, 'machines': 10, 'think?’': 11, 'This': 12, 'should': 13, 'begin': 14, 'with': 15, 'definitions': 16, 'of': 17, 'meaning': 18, 'terms': 19, '‘machine’': 20, 'and': 21, '‘think’.': 22, 'A': 23, 'computer': 24, 'would': 25, 'deserve': 26, 'be': 27, 'called': 28, 'intelligent': 29, 'if': 30, 'it': 31, 'could': 32, 'deceive': 33, 'a': 34, 'human': 35, 'into': 36, 'believing': 37, 'that': 38, 'was': 39, 'human.': 40, 'Be': 41, 'alone,': 42, 'is': 43, 'secret': 44, 'invention;': 45, 'when': 46, 'ideas': 47, 'are': 48, 'born.': 49, 'don`t': 50, 'care': 51, 'they': 52, 'stole': 53, 'my': 54, 'idea': 55, '.': 56, 'have': 57, 'any': 58, 'their': 59, 'own.': 60, 'If': 61, 'you': 62, 'can’t': 63, 'explain': 64, 'six-year-old,': 65, 'probably': 66, 'don’t': 67, 'understand': 68, 'yourself': 69, 'am': 70, 'enough': 71, 'an': 72, 'artist': 73, 'draw': 74, 'freely': 75, 'upon': 76, 'i

In [5]:
def word_to_index(tok_sentence):
  input_w2i = [[vocab_word2index[word] for word in tok_sentence[i]] for i in range(len(tok_sentence))]
  return input_w2i

w2i_input_en = word_to_index(tok_input_en)
w2i_input_ko = word_to_index(tok_input_ko)

print(w2i_input_en)
print(w2i_input_ko)

[[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 7, 18, 17, 7, 19, 20, 21, 22], [23, 24, 25, 26, 5, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 31, 39, 40], [41, 42, 38, 43, 7, 44, 17, 45, 27, 42, 38, 43, 46, 47, 48, 49], [3, 50, 51, 38, 52, 53, 54, 55, 56, 56, 3, 51, 38, 52, 50, 57, 58, 17, 59, 60], [61, 62, 63, 64, 31, 5, 34, 65, 62, 66, 67, 68, 31, 69], [3, 70, 71, 17, 72, 73, 5, 74, 75, 76, 54, 77, 78, 43, 79, 80, 81, 82, 83, 43, 84, 78, 85, 7, 86]]
[[87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105], [106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 90, 116, 117, 118, 119, 120, 121, 122, 123, 124], [125, 126, 127, 128, 129, 125, 130, 131, 127, 132, 133, 115, 134, 135], [87, 136, 137, 138, 139, 140, 141, 142, 87, 136, 143, 144, 94, 145, 146, 147, 148, 149], [150, 151, 152, 90, 153, 154, 155, 156, 157, 158, 159, 160], [87, 137, 161, 162, 163, 90, 159, 164, 165, 135, 166, 167, 168, 169, 170, 171, 166, 172, 173]]


In [6]:
def index_to_word(tok_sentence):
  input_i2w = [[vocab_index2word[word] for word in tok_sentence[i]] for i in range(len(tok_sentence))]
  return input_i2w

i2w_input_en = index_to_word(w2i_input_en)
i2w_input_ko = index_to_word(w2i_input_ko)

print(i2w_input_en)
print(i2w_input_ko)

[['I', 'PROPOSE', 'to', 'consider', 'the', 'question,', '‘Can', 'machines', 'think?’', 'This', 'should', 'begin', 'with', 'definitions', 'of', 'the', 'meaning', 'of', 'the', 'terms', '‘machine’', 'and', '‘think’.'], ['A', 'computer', 'would', 'deserve', 'to', 'be', 'called', 'intelligent', 'if', 'it', 'could', 'deceive', 'a', 'human', 'into', 'believing', 'that', 'it', 'was', 'human.'], ['Be', 'alone,', 'that', 'is', 'the', 'secret', 'of', 'invention;', 'be', 'alone,', 'that', 'is', 'when', 'ideas', 'are', 'born.'], ['I', 'don`t', 'care', 'that', 'they', 'stole', 'my', 'idea', '.', '.', 'I', 'care', 'that', 'they', 'don`t', 'have', 'any', 'of', 'their', 'own.'], ['If', 'you', 'can’t', 'explain', 'it', 'to', 'a', 'six-year-old,', 'you', 'probably', 'don’t', 'understand', 'it', 'yourself'], ['I', 'am', 'enough', 'of', 'an', 'artist', 'to', 'draw', 'freely', 'upon', 'my', 'imagination.', 'Imagination', 'is', 'more', 'important', 'than', 'knowledge.', 'Knowledge', 'is', 'limited.', 'Imagin

In [7]:
def basic_tokenizer_space(sentence, flag): # add flag.
  if 'BOS'== flag:
    tok_sentence = [[word for word in ['<BOS>'] + sentence[j].split(' ')] for j in range(len(sentence))]
  else:
    tok_sentence = [[word for word in sentence[j].split(' ') + ['<EOS>']] for j in range(len(sentence))]
  return tok_sentence

def add_eos(sentence):
  tok_input = basic_tokenizer_space(sentence, 'EOS')
  return (tok_input)

def add_bos(sentence):
  tok_input = basic_tokenizer_space(sentence, 'BOS')
  return (tok_input)

encoder_input = add_eos(input_en) # 
decoder_input = add_bos(input_ko) #
decoder_target = add_eos(input_ko) # for teacher forcing

print(encoder_input)
print(decoder_input)
print(decoder_target)

[['I', 'PROPOSE', 'to', 'consider', 'the', 'question,', '‘Can', 'machines', 'think?’', 'This', 'should', 'begin', 'with', 'definitions', 'of', 'the', 'meaning', 'of', 'the', 'terms', '‘machine’', 'and', '‘think’.', '<EOS>'], ['A', 'computer', 'would', 'deserve', 'to', 'be', 'called', 'intelligent', 'if', 'it', 'could', 'deceive', 'a', 'human', 'into', 'believing', 'that', 'it', 'was', 'human.', '<EOS>'], ['Be', 'alone,', 'that', 'is', 'the', 'secret', 'of', 'invention;', 'be', 'alone,', 'that', 'is', 'when', 'ideas', 'are', 'born.', '<EOS>'], ['I', 'don`t', 'care', 'that', 'they', 'stole', 'my', 'idea', '.', '.', 'I', 'care', 'that', 'they', 'don`t', 'have', 'any', 'of', 'their', 'own.', '<EOS>'], ['If', 'you', 'can’t', 'explain', 'it', 'to', 'a', 'six-year-old,', 'you', 'probably', 'don’t', 'understand', 'it', 'yourself', '<EOS>'], ['I', 'am', 'enough', 'of', 'an', 'artist', 'to', 'draw', 'freely', 'upon', 'my', 'imagination.', 'Imagination', 'is', 'more', 'important', 'than', 'knowle

In [8]:
encoder_input = word_to_index(encoder_input)
decoder_input = word_to_index(decoder_input)
decoder_target = word_to_index(decoder_target)

In [9]:
print(encoder_input)
print(decoder_input)
print(decoder_target)

[[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 7, 18, 17, 7, 19, 20, 21, 22, 2], [23, 24, 25, 26, 5, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 31, 39, 40, 2], [41, 42, 38, 43, 7, 44, 17, 45, 27, 42, 38, 43, 46, 47, 48, 49, 2], [3, 50, 51, 38, 52, 53, 54, 55, 56, 56, 3, 51, 38, 52, 50, 57, 58, 17, 59, 60, 2], [61, 62, 63, 64, 31, 5, 34, 65, 62, 66, 67, 68, 31, 69, 2], [3, 70, 71, 17, 72, 73, 5, 74, 75, 76, 54, 77, 78, 43, 79, 80, 81, 82, 83, 43, 84, 78, 85, 7, 86, 2]]
[[1, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105], [1, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 90, 116, 117, 118, 119, 120, 121, 122, 123, 124], [1, 125, 126, 127, 128, 129, 125, 130, 131, 127, 132, 133, 115, 134, 135], [1, 87, 136, 137, 138, 139, 140, 141, 142, 87, 136, 143, 144, 94, 145, 146, 147, 148, 149], [1, 150, 151, 152, 90, 153, 154, 155, 156, 157, 158, 159, 160], [1, 87, 137, 161, 162, 163, 90, 159, 164, 165, 135, 166, 167, 168, 169, 170, 171, 166, 172,

In [10]:
def add_padding(index_sentence):
  padd_input = []
  for s in index_sentence:
      padd_input.append(torch.LongTensor(s))
  inputs = torch.nn.utils.rnn.pad_sequence(padd_input, batch_first=True, padding_value=0)
  
  return inputs

encoder_input = add_padding(encoder_input)
decoder_input = add_padding(decoder_input)
decoder_target = add_padding(decoder_target)
print(encoder_input.size())
print(encoder_input)
print(decoder_input.size())
print(decoder_input)
print(decoder_target.size())
print(decoder_target)


torch.Size([6, 26])
tensor([[ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,  7, 18, 17,
          7, 19, 20, 21, 22,  2,  0,  0],
        [23, 24, 25, 26,  5, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 31,
         39, 40,  2,  0,  0,  0,  0,  0],
        [41, 42, 38, 43,  7, 44, 17, 45, 27, 42, 38, 43, 46, 47, 48, 49,  2,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 3, 50, 51, 38, 52, 53, 54, 55, 56, 56,  3, 51, 38, 52, 50, 57, 58, 17,
         59, 60,  2,  0,  0,  0,  0,  0],
        [61, 62, 63, 64, 31,  5, 34, 65, 62, 66, 67, 68, 31, 69,  2,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 3, 70, 71, 17, 72, 73,  5, 74, 75, 76, 54, 77, 78, 43, 79, 80, 81, 82,
         83, 43, 84, 78, 85,  7, 86,  2]])
torch.Size([6, 21])
tensor([[  1,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
         100, 101, 102, 103, 104, 105,   0],
        [  1, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,  90, 116, 117,
         118, 119,

In [11]:
class Encoder(nn.Module):
  def __init__(self, vocabulary_size, hidden_size):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(vocabulary_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

  def forward(self, sentence, h):
    x = self.embedding(sentence) #[batch, seq_length, embedding_hidden]
    encoder_out, h_out  = self.gru(x, h)
    return encoder_out, h_out

  def init_hidden(self, batch_size):
    h0 = torch.zeros(1, batch_size, self.hidden_size)# [num_layer, batch_size, hidden_size]
    return h0

In [12]:
vocabulary_size = len(vocab_index2word)
hidden_size = 256
batch_size = encoder_input.size(0)

seq2seq_encoder = Encoder(vocabulary_size, hidden_size)

In [13]:
h0 = seq2seq_encoder.init_hidden(batch_size)
encoder_out, encoder_h = seq2seq_encoder(encoder_input, h0)
print(encoder_out)
print(encoder_h)

tensor([[[ 1.0578e-01, -4.2334e-01,  9.1939e-02,  ...,  1.1392e-01,
          -1.9570e-01,  3.0483e-02],
         [-1.2534e-01, -5.7273e-01,  2.5832e-01,  ..., -1.4612e-01,
           1.0154e-01, -1.3361e-01],
         [-4.3737e-02, -1.3441e-01, -5.1799e-02,  ..., -3.7719e-02,
           4.3107e-01, -1.6470e-02],
         ...,
         [-3.2394e-01, -1.7356e-01,  4.0364e-01,  ..., -1.0096e-01,
          -2.4934e-01, -2.8686e-01],
         [-1.9364e-01, -2.6060e-01,  3.1201e-01,  ..., -7.1571e-02,
          -1.9597e-01, -1.1875e-01],
         [-9.3884e-02, -3.3563e-01,  2.4027e-01,  ..., -4.5202e-02,
          -2.0424e-01, -1.1985e-04]],

        [[ 1.6644e-01, -1.5644e-01,  4.9700e-02,  ..., -2.0275e-01,
           2.7537e-03, -2.2857e-01],
         [-1.9407e-01, -6.0694e-01, -2.8372e-01,  ..., -4.7299e-01,
          -1.9072e-01,  7.6128e-02],
         [ 8.1878e-02, -2.3193e-02, -1.8616e-01,  ..., -4.6087e-01,
           2.0897e-01,  1.9428e-01],
         ...,
         [ 2.0073e-02, -3

In [14]:
class Decoder(nn.Module):
  def __init__(self, vocabulary_size, hidden_size):
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(vocabulary_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, vocabulary_size)
    
  def forward(self, sentence, encoder_h):
    x = self.embedding(sentence.view(-1, 1)) #[batch, seq_length, embedding_hidden]
    decoder_out, h_out  = self.gru(x, encoder_h)
    decoder_out = self.fc(decoder_out)
    decoder_out = F.softmax(decoder_out, dim=-1)
    return decoder_out, h_out


In [15]:
seq2seq_decoder = Decoder(vocabulary_size, hidden_size)

In [16]:
criterion = nn.CrossEntropyLoss(ignore_index=-1)

seq_length = decoder_input.size(1)
h_in = encoder_h
loss = 0
for i in range(seq_length): # teacher forcing.
  decoder_out, h_out = seq2seq_decoder(decoder_input[:,i], h_in)
  h_in = h_out
  mask = torch.zeros_like(decoder_target[:,i]) # <pad>로 채워진 부분 loss를 무시하기 위한 mask.
  mask = torch.eq(decoder_target[:,i], mask)
  decoder_target[:,i].masked_fill_(mask, -1)
  loss += criterion(decoder_out.squeeze(1), decoder_target[:,i])
total_loss = loss/seq_length
print(total_loss)

tensor(5.1593, grad_fn=<DivBackward0>)


In [17]:
def train(encoder_input, decoder_input, decoder_target, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
 
  batch_size = encoder_input.size(0)
  seq_length = decoder_input.size(1)
  
  h0 = encoder.init_hidden(batch_size)
  encoder_out, encoder_h = encoder(encoder_input, h0)

  h_in = encoder_h
  loss = 0
  for i in range(seq_length): # teacher forcing.
    decoder_out, h_out = decoder(decoder_input[:,i].contiguous(), h_in)
    h_in = h_out
    mask = torch.zeros_like(decoder_target[:,i].contiguous()) # <pad>로 채워진 부분 loss를 무시하기 위한 mask.
    mask = torch.eq(decoder_target[:,i].contiguous(), mask)
    decoder_target[:,i].contiguous().masked_fill_(mask, -1)
    loss += criterion(decoder_out.squeeze(1), decoder_target[:,i].contiguous())

  total_loss = loss/seq_length
  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()
  total_loss.backward()
  encoder_optimizer.step()
  decoder_optimizer.step()
  return total_loss

In [18]:
def main():
  vocabulary_size = len(vocab_index2word)
  hidden_size = 256
  learning_rate = 0.0001
  epoch = 50
  criterion = nn.CrossEntropyLoss(ignore_index=-1)

  encoder = Encoder(vocabulary_size, hidden_size)
  decoder = Decoder(vocabulary_size, hidden_size)

  encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
  decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

  for i in range(epoch):
    loss = train(encoder_input, decoder_input, decoder_target, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
    print(loss)

In [19]:
main()

tensor(5.1591, grad_fn=<DivBackward0>)
tensor(5.1589, grad_fn=<DivBackward0>)
tensor(5.1588, grad_fn=<DivBackward0>)
tensor(5.1586, grad_fn=<DivBackward0>)
tensor(5.1584, grad_fn=<DivBackward0>)
tensor(5.1583, grad_fn=<DivBackward0>)
tensor(5.1581, grad_fn=<DivBackward0>)
tensor(5.1579, grad_fn=<DivBackward0>)
tensor(5.1577, grad_fn=<DivBackward0>)
tensor(5.1575, grad_fn=<DivBackward0>)
tensor(5.1573, grad_fn=<DivBackward0>)
tensor(5.1571, grad_fn=<DivBackward0>)
tensor(5.1568, grad_fn=<DivBackward0>)
tensor(5.1566, grad_fn=<DivBackward0>)
tensor(5.1563, grad_fn=<DivBackward0>)
tensor(5.1560, grad_fn=<DivBackward0>)
tensor(5.1557, grad_fn=<DivBackward0>)
tensor(5.1554, grad_fn=<DivBackward0>)
tensor(5.1550, grad_fn=<DivBackward0>)
tensor(5.1546, grad_fn=<DivBackward0>)
tensor(5.1541, grad_fn=<DivBackward0>)
tensor(5.1536, grad_fn=<DivBackward0>)
tensor(5.1531, grad_fn=<DivBackward0>)
tensor(5.1525, grad_fn=<DivBackward0>)
tensor(5.1518, grad_fn=<DivBackward0>)
tensor(5.1510, grad_fn=<D